In [135]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import tensorflow as tf

from tensorflow.keras.datasets import mnist

from tensorflow.keras import Model,layers

In [136]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train = tf.cast(X_train/255.0,tf.float32)
y_train = tf.one_hot(y_train,depth=10)#10分类问题
X_test = tf.cast(X_test/255,tf.float32)
y_test = tf.one_hot(y_test,depth=10)

# 将数据X_train,X_test reshape一下，数据形状满足神经网络要求
X_train = tf.reshape(X_train,shape = [-1,28,28,1])
X_test = tf.reshape(X_test,shape = [-1,28,28,1])

# X_train 60000个，重复了100次，每次取512
data_train = tf.data.Dataset.from_tensor_slices((X_train,y_train)).repeat(100).shuffle(2000).batch(128).prefetch(1)
# X_test 10000个，可以取5次，重复50次，250次
data_test = tf.data.Dataset.from_tensor_slices((X_test,y_test)).repeat(50).shuffle(2000).batch(128).prefetch(1)

### 声明模型、损失、准确率、优化算法Adam

In [137]:
class Cnn(Model):
    def __init__(self):
        super(Cnn,self).__init__()
#         第一层卷积网络
        self.conv1 = layers.Conv2D(filters = 32,kernel_size=(3,3),padding='same',activation='relu')
#      strides: Integer, tuple of 2 integers, or None.
#     Strides values.
#     If None, it will default to `pool_size`.
        self.pool1 = layers.MaxPool2D(padding='same')
    
#         第二层卷积网络
        self.conv2 = layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
        self.pool2 = layers.MaxPool2D(padding='same')
        
#         第三层卷积网络
        self.conv3 = layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
        self.pool3 = layers.MaxPool2D(padding='same')
        
#         全连接层，self.pool3返回的数据是4维，降维
        self.flattern = layers.Flatten() # reshape
        self.fc = layers.Dense(1024,activation=tf.nn.relu)
        
#         dropout，防止过拟合，掩盖一些神经元，不让其工作，系数表少了
#         普通算法，sklearn中通过L1、L2正则化，实现防止过拟合
#          训练时，dropout，测试的时候，依然是完全的网络
        self.dropout = layers.Dropout(rate = 0.5)
#         输出层
        self.out = layers.Dense(10)#Dense相当于，矩阵运算
    def call(self,X,is_training = False):
        X = self.conv1(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = self.pool2(X)
#         X = self.conv3(X)
#         X = self.pool3(X)
        X = self.flattern(X)#reshape
        X = self.fc(X)
#         X = self.dropout(X,training = is_training)
        y_pred = self.out(X)#输出层，模型的结果，预测，此处，没有进行softmax。神经网络中称为：logits
        if not is_training:# 测试
            y_pred = tf.nn.softmax(y_pred)
        return y_pred

cnn = Cnn()

#损失
def cross_entropy(y_true,y_pred):
    y_pred = tf.clip_by_value(y_pred,1e-9,1.0)
#     logits 表示，模型的输出结果（没有进行softmax的结果）
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_true,logits=y_pred)
    return loss

# 准确率
def accuracy(y_true,y_pred):
    y_true = tf.argmax(y_true,axis = -1)
    y_pred = tf.argmax(y_pred,axis = -1)
    acc = tf.reduce_mean(tf.cast(tf.equal(y_true,y_pred),dtype=tf.float16))
    return acc

# 优化，Adam优化算法，是在SGD的基础上，进行的优化
# Adam optimization is a stochastic gradient descent method that is based on
# adaptive estimation of first-order and second-order moments.
adam = tf.optimizers.Adam(learning_rate=0.0005)

### 定义优化过程

In [138]:
def run_optimizer(X_train,y_train):
    with tf.GradientTape() as g:
        y_pred = cnn(X_train,is_training = True)
        loss = cross_entropy(y_train,y_pred)
    gradients = g.gradient(loss,cnn.trainable_variables)
    adam.apply_gradients(zip(gradients,cnn.trainable_variables))

### for循环进行训练

In [139]:
for i,(X_train,y_train) in enumerate(data_train.take(100),101):
    run_optimizer(X_train,y_train)
    if i %10 == 0:
        for (X_test,y_test) in data_test.take(1):
            y_pred = cnn(X_test,is_training = False)
            acc = accuracy(y_test,y_pred)
            y_pred2 = cnn(X_train,is_training = False)
            acc2 = accuracy(y_train,y_pred2)
            print('执行次数：%d。训练数据准确率：%0.4f。测试数据准确率：%0.4f'%(i,acc2,acc))

执行次数：110。训练数据准确率：0.4766。测试数据准确率：0.4453
执行次数：120。训练数据准确率：0.4766。测试数据准确率：0.5391
执行次数：130。训练数据准确率：0.5312。测试数据准确率：0.5312
执行次数：140。训练数据准确率：0.5781。测试数据准确率：0.5703
执行次数：150。训练数据准确率：0.5312。测试数据准确率：0.5391
执行次数：160。训练数据准确率：0.5859。测试数据准确率：0.5391
执行次数：170。训练数据准确率：0.6094。测试数据准确率：0.5703
执行次数：180。训练数据准确率：0.6172。测试数据准确率：0.5625
执行次数：190。训练数据准确率：0.5469。测试数据准确率：0.6328
执行次数：200。训练数据准确率：0.6562。测试数据准确率：0.5859
